In [2]:
!pip install deap --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for deap: filename=deap-1.4.1-cp311-cp311-linux_x86_64.whl size=133279 sha256=9416527e61dae23addc86d12aa5d3c993b597e1c9c4dfff2b1caf8760d13aedf
  Stored in directory: /home/pedrov/.cache/pip/wheels/f8/64/b8/65eacfbff3024ae2e2beb22e691d5c8abb89fbd863b8049b5f
Successfully built deap


In [1]:
import random
from deap import base, creator, tools, algorithms

# Definir o tipo Fitness e Indivíduo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=10)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Função de Avaliação
def evalOneMax(individual):
    return (sum(individual),)

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Função para implementar RCE
def rce(population, elite_size):
    # Selecionar elite
    elite = tools.selBest(population, elite_size)

    # Repopular o restante da população
    rest = toolbox.select(population, len(population) - elite_size)
    rest = list(map(toolbox.clone, rest))

    # Aplicar crossover e mutação
    for child1, child2 in zip(rest[::2], rest[1::2]):
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values
    for mutant in rest:
        toolbox.mutate(mutant)
        del mutant.fitness.values
    
    # Retornar a nova população
    return elite + rest

# Função principal
def main():
    random.seed(64)
    pop = toolbox.population(n=50)
    elite_size = 10

    for gen in range(50):
        pop = rce(pop, elite_size)
        fitnesses = list(map(toolbox.evaluate, pop))
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
    return pop

pop = main()
best_ind = tools.selBest(pop, 1)[0]
print("Melhor Indivíduo:", best_ind, best_ind.fitness.values)


ModuleNotFoundError: No module named 'deap'

Passos para Selecionar e Exibir Elementos de Elite

1) Executar o Algoritmo Evolutivo: Primeiro, você roda o algoritmo evolutivo como já definido anteriormente.

2) Selecionar Elementos de Elite: Após a execução do algoritmo, selecione um número desejado de melhores indivíduos da população final.

3) Exibir os Elementos de Elite: Exiba os vetores (genótipos) e suas respectivas avaliações (fitness) dos elementos selecionados.



In [ ]:
# ... [restante do código anterior]

# Função principal
def main():
    random.seed(64)
    pop = toolbox.population(n=50)
    elite_size = 10

    for gen in range(50):
        pop = rce(pop, elite_size)
        fitnesses = list(map(toolbox.evaluate, pop))
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
    return pop

# Executar o algoritmo
pop = main()

# Selecionar os melhores indivíduos da população
num_elite = 5  # número de elementos de elite a serem selecionados
elite_individuals = tools.selBest(pop, num_elite)

# Exibir os elementos de elite
print("Elementos de Elite:")
for i, ind in enumerate(elite_individuals, 1):
    print(f"{i}. Vetor: {ind}, Fitness: {ind.fitness.values}")


## Como fazer uma minimação para  30% dos melhores aptidoes

In [ ]:
import random
from deap import base, creator, tools, algorithms

# Define o problema para maximizar a função de fitness
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
N = 10  # Número máximo que pode aparecer no vetor

# Atributo: cada elemento do vetor varia de 0 a N
toolbox.register("attr_int", random.randint, 0, N)

# Estrutura do indivíduo: um vetor com M elementos
M = 4  # Tamanho do vetor de entrada
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=M)

# População: uma lista de indivíduos
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Função de avaliação: soma dos elementos do vetor
def evalSum(individual):
    return (sum(individual),)

toolbox.register("evaluate", evalSum)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=N, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Função principal para rodar o algoritmo genético
def main():
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1, similar=lambda x: x)  # Hall of Fame para armazenar o melhor indivíduo

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(x)/len(x))
    stats.register("min", min)
    stats.register("max", max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40,
                                   stats=stats, halloffame=hof, verbose=True)

    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()
    print("Melhor indivíduo é: ", hof[0], "com a soma: ", hof[0].fitness.values[0])


In [ ]:
# ... [restante do código anterior]

# Função para checar se há diferença suficiente entre os indivíduos
def delta_difference(individuals, delta):
    unique_individuals = [individuals[0]]  # Começa com o melhor indivíduo
    for ind in individuals[1:]:
        if all(abs(val_i - val_j) >= delta for i, val_i in enumerate(ind) 
                                           for j, val_j in enumerate(unique_individuals[-1]) 
                                           if i == j):
            unique_individuals.append(ind)
    return unique_individuals

# Função principal modificada para minimização
def main():
    # ... [restante do código da função main]
    
    # Selecionar os 30% melhores indivíduos
    top_30_percent = int(len(pop) * 0.3)
    best_individuals = tools.selBest(pop, top_30_percent)
    
    # Filtrar os melhores indivíduos para garantir diferença mínima delta
    delta = 3
    unique_individuals = delta_difference(best_individuals, delta)
    
    return unique_individuals

if __name__ == "__main__":
    unique_individuals = main()
    print("Indivíduos únicos com delta >= 3:")
    for ind in unique_individuals:
        print(ind, ind.fitness.values)
